## Load packages

In [1]:
#!python -m spacy download en_core_web_lg
# !conda install spacy download en_core_web_lg
# !python -m spacy validate
import pandas as pd
import spacy.displacy as displacy
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
import random

## Working with NER

In [2]:
# df = pd.read_csv('ubereats_cleaned.csv', header=None)
# type(df)

In [3]:
# df.head()

In [4]:
# df[0].head(50)

In [5]:
# data = list(df[0].head(50))

In [6]:
# def find_nth(haystack, needle, n):
#     start = haystack.find(needle)
#     while start >= 0 and n > 1:
#         start = haystack.find(needle, start+len(needle))
#         n -= 1
#     return start

## Train data

In [7]:
random.seed(42)


In [8]:
train_list = []

In [9]:
# Reading in first 10000 lines of the text file
count = 0
num_lines = 10000
with open('review_train', 'r') as f:
    for line in f:
        if count == num_lines:
            break
        print(line, sep = '\n')
        train_list.append(line)
        count +=1

my favorite dish is the  <START:food>  fried chicken eggs benedict <END>  

i can't imagine eating that giant  <START:food>  cream puff <END>  

All the food we had was delicious i love how they leave a carafe of  <START:food> coffee <END>  on the table 

Only went here once about a year a half ago, they had great  <START:food>  pancakes <END>  

they have  <START:food> doughnuts <END>  the size of your head 

 <START:food> eclairs <END>  the size of your arm and  <START:food>  cream puffs <END>  that are about 10 high 

On our most recent trip in the area, stopped at the gas station attached to the restaurant fo some quick  <START:food>  snacks <END>  of course bakery 

If you are driving by see the sign for the pine cone restaurant Take the next off  <START:food> ramp <END>  stop it 

it was near dinner time so i ordered stuffed  <START:food>  chicken breast <END>  

for the new year the special was you can eat  <START:food> pancakes <END>  for 212, i added a breakfast  <START:food> 


the  <START:food>  shrimp <END>  was not overcooked the noodes were packed again with tons of veggies 

they have the best pork  <START:food>  egg rolls <END>  i've ever had 

they make a big claim that if you don't see it on their menu they'll find a way to make it, however that didn't hold true when i asked about cantonese pan fried  <START:food> noodles <END>  an item that seems near impossible to find in this town 

After i ordered the general tso's chicken, the woman turned on the fryer and, without waiting for the oil to heat she put in the  <START:food>  chicken <END>  my  <START:food>  crab rangoon <END>  

another thing that i noticed was that the  <START:food>  chicken <END>  that she put in the fryer was sitting on the counter in the heat 

If you live on the east side do yourself a favor get the happy family, lo mein, fried  <START:food>  pork <END>  wontons i was a little thrown when i first moved to the east side of madison drove past this place 

Sometimes you have to m

Quick review  <START:food>  lentil <END>  soup 

beautiful  <START:food>  saffron <END>  color from the cooked down red lentils, just the right mix of  <START:food>  cumin <END>  and lemon, a back end of heat, I'm guessing from aleppo pepper 

i like roasted red bell pepper, and i've had  <START:food>  dips <END>  and purees made of them, but i've never sought it out 

a smack of roasted  <START:food>  red pepper <END>  is quickly deepened by a sweet hit of  <START:food>  pomegranate <END>  and walnuts, ubelievably complex and wonderful 

 <START:food> falafel <END>  

Love the falafel, i prefer  <START:food>  tahini <END>  sauce, these come with a yogurt  <START:food>  mint sauce <END>  

i confess that for my leftovers I'm making some  <START:food>  tahini <END>  sauce 

 <START:food> hummus <END>  

you know,  <START:food> hummus <END>  is always a crap shoot for me 

i'll try it again, because my own  <START:food>  hummus <END>  has on and off days 

i added some  <START:food>  lem


i doubt any of the  <START:food>  fish <END>  is fresh, perhaps on the weekends, it is 

all meals come with a  <START:food>  salad <END>  

Get the  <START:food>  blue cheese <END>  dressing 

you can also add  <START:food>  blue cheese <END>  crumbles 

 <START:food> soup <END>  was  <START:food>  bacon <END>  cheeseburger chowder 

toby's has one of the best  <START:food>  fish <END>  fry's around which makes this place super packed on fridays 

my husband loves the  <START:food>  steaks <END>  

i usually order a variety of their  <START:food>  seafood <END>  

 <START:food> salads <END>  are always fresh 

Relax with one of their signature  <START:food>  key lime <END>  martinisbetter yet, make it two 

toby's has the best  <START:food>  fish <END>  fry in the madison area without a doubt 

i had read about  <START:food>  key lime <END>  martinis being their specialty drink i ordered one it was 

the  <START:food>  fish <END>  was 

i ordered the  <START:food>  cod <END>  my dad 

my  <START:food>  chicken <END>  sandwich was an average  <START:food>  chicken <END>  sandwich 

we were there unexpectedly during happy hour, which i think was 5 7my daughter i had several of the 6 appetizers my husband had the ragin'  <START:food>  cajun <END>  chicken sandwich which was so huge that at first he thought it was battered was actually a lot of 

i ordered the  <START:food>  chicken <END>  sandwich with sprouts avocado 

it came without any other  <START:food>  condiments <END>  

this was a problem since it was the driest piece of  <START:food> chicken <END>  i had ever attempted to eat 

Luckily the waitress hadn't cleared the wing plate i was able to use the ranch to get 12 of my  <START:food>  sandwich <END>  down 

Speaking of my husband, he ordered a  <START:food>  burger <END>  that also came without  <START:food> condiments <END>  

So, remember if like something besides  <START:food> catsup <END>  on your  <START:food>  sandwiches <END>  you'd better ask, stay 


Before my entree arrived i was given the standard  <START:food>  chips <END>  

Before i could get the bottom of the  <START:food>  chip <END>  bowl my dinner was already on it's way over 

the  <START:food>  enchilada <END>  dinner left much to be desired for the price i paid 

incredible  <START:food>  salsa <END>  

this place was closest to my hotel i almost skipped it for a  <START:food>  taco <END>  bar place because of these reviews about the food and  <START:food>  salsa <END>  

 <START:food> salsa <END>  they bring you two types 

 <START:food> fajitas <END>  were good 

 <START:food> enchiladas <END>  were good 

 <START:food> rice <END>  was 

flour  <START:food>  tortillas <END>  seemed homemade, but something was off i think we got older ones stale make them fresh change your recipe something there needs a change 

i give the higher than average star rating based on service the  <START:food>  salsa <END>  

the  <START:food>  salsa <END>  is fresh and good 

made chunk s

i am in lovewith zoe's grilled  <START:food>  feta <END>  sandwich 

everytime my co workers i go for lunch, i always tell myself i am going to try something else on the menuand then i don't because i don't want to cheat on the grilled  <START:food>  feta <END>  

my  <START:food>  sandwich <END>  is always toasted perfectly the  <START:food>  tomato <END>  on the  <START:food>  sandwich <END>  helps tone down the  <START:food>  feta <END>  just a touch 

the  <START:food>  pasta salad <END>  is my favorite of the sides to get 

i tried the  <START:food>  potato salad <END>  once was not wowed by it at 

As a warning, if you don't like  <START:food>  feta <END>  cheese, this may not be the place for you 

Was very satisfied with my meal of 2  <START:food>  chicken <END>  kabobs, rice and side salad accompanied with a couple of pieces of pita 

the tables are stocked with a bottle of what appears to be their homemade  <START:food>  vinaigrette <END>  dressing, also very tasty 

i always

In [10]:
len(train_list)

10000

In [11]:
# Key words in text
start = '<START:food>'
end = '<END>'

replace_vals = {start:'', end:''}
# Function to replace keywords with ''
def replace_all(text, dic):
    for i,j in dic.items():
        text = text.replace(i,j)
    return text

In [12]:
# test = train_list[5]
# test_food_items = []
# print(test)

In [13]:
# split = test.split(start)
# food_items = []
# for i in range(0,len(split)-1):
#     test_food_items.append(split[i+1].split(end)[0].strip())

In [14]:
# print(test_food_items)

In [15]:
# test = replace_all()

### GOD TIER CELL - Need to modularize later

In [17]:
TRAIN_DATA = []
# Iterate through each review
for sentence in train_list:
    # Get rid of leading/trailing spaces plus '\n'
    sentence = sentence.strip()
    
    # Code to get the all values inside '<START:food>' and '<END>'
    split = sentence.split(start)
    food_items = []
    for i in range(0,len(split)-1):
        food_items.append(split[i+1].split(end)[0].strip())
    
#     for val in food_items:
#         print(food_items)
    # Removing all marker Keywords, removing extra spaces, and sentence lowercase
    sentence = replace_all(sentence, replace_vals)
    sentence = ' '.join(sentence.split())
    sentence = sentence.lower()
    #train_list[ind] = train_list[ind].strip()
    #print(sentence)
    
    # Creating the format necessary for spacy custom NER
    entity_list = []
    for val in food_items:
        # Find the starting index of word
        ind_start = sentence.find(val)
        ind_end = ind_start + len(val)
        entity_list.append((ind_start, ind_end, 'FOOD') )
        #
    
    new_format = (sentence, {'entities': entity_list})
    TRAIN_DATA.append(new_format)

In [19]:
TRAIN_DATA

[('my favorite dish is the fried chicken eggs benedict',
  {'entities': [(24, 51, 'FOOD')]}),
 ("i can't imagine eating that giant cream puff",
  {'entities': [(34, 44, 'FOOD')]}),
 ('all the food we had was delicious i love how they leave a carafe of coffee on the table',
  {'entities': [(68, 74, 'FOOD')]}),
 ('only went here once about a year a half ago, they had great pancakes',
  {'entities': [(60, 68, 'FOOD')]}),
 ('they have doughnuts the size of your head',
  {'entities': [(10, 19, 'FOOD')]}),
 ('eclairs the size of your arm and cream puffs that are about 10 high',
  {'entities': [(0, 7, 'FOOD'), (33, 44, 'FOOD')]}),
 ('on our most recent trip in the area, stopped at the gas station attached to the restaurant fo some quick snacks of course bakery',
  {'entities': [(105, 111, 'FOOD')]}),
 ('if you are driving by see the sign for the pine cone restaurant take the next off ramp stop it',
  {'entities': [(82, 86, 'FOOD')]}),
 ('it was near dinner time so i ordered stuffed chicken br

In [ ]:
# test = 'eclairs the size of your arm and cream puffs that are about 10 high'
# test[33:44]

In [ ]:

#s = 'for the new year <START:food> turkey tacos <END>  the special was you can eat  <START:food> pancakes <END>  for 212, i added a breakfast  <START:food>  sausage patty <END>'

In [ ]:
# def find_locations(haystack, needle):
#     locations = []
#     start = haystack.find(needle)
#     locations.append(start)
#     while start >= 0:
#         start = haystack.find(needle, start+len(needle))
#         if start < 0:
#             break
#         locations.append(start)
#     return locations

In [ ]:
# test = 'for the new year the special was you can eat  <START:food> pancakes <END>  for 212, i added a breakfast  <START:food>  sausage patty <END>'

In [ ]:
# find_locations(test, '<START:food>')

In [ ]:
# sentences = [
#     'I like to eat __ at this joint',
#     'The __ was fire', 
#     '__ are amazing, you have to try it!', 
#     'Everytime I visit, I always get __',
#     'My friends and I are never getting __ again',
#     'This place always messes up my order of __',
#     'I think that __ is the best thing on the menu'
# ]


In [ ]:
# TRAIN_DATA = []

# target_sent = ''
# nbr_of_sentences = len(sentences)

# for ind, val in enumerate(data):
    
#     for i in range(nbr_of_sentences):
#         if ind % nbr_of_sentences == i:
#             target_sent = sentences[i]
#             ind_start = target_sent.find('__')
#             ind_end = ind_start + len(val)
#             target_sent = target_sent.replace('__', val)
#             break
        
        
        
# #     random_sentence = gen.sentence()
# #     count = random_sentence.count(' ')
# #     random_space = randrange(count)
#     #index = find_nth(random_sentence, ' ', random_space)
#     #final_sentence = random_sentence[: index]+ ' ' + val + random_sentence[index:]
#     #final_sentence.replace('\n', '')
#     target_sent = target_sent.lower()
#     #final_sentence = re.sub(r'[^a-z ]', '', final_sentence)
#     new_format = (target_sent, {'entities':[(ind_start, ind_end, 'FOOD')]})
#     TRAIN_DATA.append(new_format)

#  #("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
# # TRAIN_DATA = [
# #     ('hot pepper chicken nuggets are best at this joint', {
# #         'entities': [(0, 26, 'FOOD')]
# #     }),
# #      ('Turkey tacos was fire', {
# #         'entities': [(0, 12, 'FOOD')]
# #     }),
# #     ('Pork Ribs are good, you have to try it!', {
# #         'entities': [(0, 9, 'FOOD')]
# #     })
# # ]


# #   'entities': [(7, 13, 'FOOD'), (18, 24, 'LOC')] - more than 1 entity in a sentence

In [ ]:
# TRAIN_DATA

In [ ]:
# # GOD TIER COPY
# TRAIN_DATA = [
#     ('I like to eat hot pepper chicken nuggets at this joint', {
#         'entities': [(14, 40, 'FOOD')]
#     }),
#      ('The Turkey tacos was fire', {
#         'entities': [(4, 16, 'FOOD')]
#     }),
#     ('Pork Ribs You have to try it!', {
#         'entities': [(0, 9, 'FOOD')]
#     })
# ]

## Define variables

In [20]:
model = None
output_dir=Path("/Users/rahul/Desktop/entity/output_dir")
n_iter=10

## Load the model

In [21]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


## Setup pipeline

In [22]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

## Train the recognizer

In [ ]:
for _, annotations in TRAIN_DATA:
    print(_, annotations)

    for ent in annotations.get('entities'):
        print(ent)
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

my favorite dish is the fried chicken eggs benedict {'entities': [(24, 51, 'FOOD')]}
(24, 51, 'FOOD')
i can't imagine eating that giant cream puff {'entities': [(34, 44, 'FOOD')]}
(34, 44, 'FOOD')
all the food we had was delicious i love how they leave a carafe of coffee on the table {'entities': [(68, 74, 'FOOD')]}
(68, 74, 'FOOD')
only went here once about a year a half ago, they had great pancakes {'entities': [(60, 68, 'FOOD')]}
(60, 68, 'FOOD')
they have doughnuts the size of your head {'entities': [(10, 19, 'FOOD')]}
(10, 19, 'FOOD')
eclairs the size of your arm and cream puffs that are about 10 high {'entities': [(0, 7, 'FOOD'), (33, 44, 'FOOD')]}
(0, 7, 'FOOD')
(33, 44, 'FOOD')
on our most recent trip in the area, stopped at the gas station attached to the restaurant fo some quick snacks of course bakery {'entities': [(105, 111, 'FOOD')]}
(105, 111, 'FOOD')
if you are driving by see the sign for the pine cone restaurant take the next off ramp stop it {'entities': [(82, 86, 'FOO

the chopped peanuts added no crunch at all, which is too bad because other than the chicken pieces i picked through the mess to find, i was hoping to find something else edible {'entities': [(12, 19, 'FOOD'), (84, 91, 'FOOD')]}
(12, 19, 'FOOD')
(84, 91, 'FOOD')
on my next visit, i ordered the firecracker noodles with chicken, as a side to my beloved wontons {'entities': [(44, 51, 'FOOD')]}
(44, 51, 'FOOD')
the red peppers had no life {'entities': [(4, 15, 'FOOD')]}
(4, 15, 'FOOD')
again, i hunted pecked for the chicken, which is consistently decent quality, well prepared white meat {'entities': [(92, 102, 'FOOD')]}
(92, 102, 'FOOD')
my wife's veggie fried rice would have been ok they ease up on the salt valvoline for this entree the vegetables were overcooked and charred in places {'entities': [(17, 27, 'FOOD'), (102, 112, 'FOOD')]}
(17, 27, 'FOOD')
(102, 112, 'FOOD')
fresh veggies should present as a bright spectrum of colors orange from the carrots, bright green from the scallions, b

she delights in the free mug of soda and balloon {'entities': [(32, 36, 'FOOD')]}
(32, 36, 'FOOD')
when our meals finally arrived, my children discovered that the insides of their burgers were very pink {'entities': [(81, 88, 'FOOD')]}
(81, 88, 'FOOD')
it was scary pink and super soft like raw hamburger generally {'entities': [(42, 51, 'FOOD')]}
(42, 51, 'FOOD')
as a side my husband's burger was to be well done was also pink, though not raw {'entities': [(23, 29, 'FOOD')]}
(23, 29, 'FOOD')
no one offered a discount, a refund, a free soda while we waited, especially not an apology {'entities': [(44, 48, 'FOOD')]}
(44, 48, 'FOOD')
ok, i heard from someone that culver's had cheese curds {'entities': [(50, 55, 'FOOD')]}
(50, 55, 'FOOD')
for those that don't know what a cheese curd is ready this wiki article {'entities': [(40, 44, 'FOOD')]}
(40, 44, 'FOOD')
so, when class got out early today, i decided that i needed to hunt for the elusive curds {'entities': [(84, 89, 'FOOD')]}
(84, 89, 'FO

## Test the trained recognizer

In [ ]:
df = pd.read_csv('reviews.csv')

In [ ]:
df.head()

In [ ]:
TEST_DATA = list(df.loc[0:3,'text'])

In [ ]:
TEST_DATA

In [ ]:
# TEST_DATA = [
#     ('Anytime I am in Chicago I always get the phones, but this time I chose to get the laptops and I loved it', {
#         'entities': [(0, 25, 'FOOD')]
#     }),
#      ('i like to eat butter chicken', {
#         'entities': [(0, 12, 'FOOD')]
#     }),
#     ('The double ka meetha is awesome here', {
#         'entities': [(0, 9, 'FOOD')]
#     })
# ]

# TEST_DATA = [
#     'The chicken eggs benedict was dope',
#     'Have to try the coffee here',
#     'OMG The eclairs here are to DIE FOR',
#     'Best snacks in town. Must try'
# ]
for text in TEST_DATA:
    doc = nlp(text)
    #print(doc)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [ ]:
# for text, _ in TRAIN_DATA:
#     doc = nlp(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

## Save model

In [ ]:
# if output_dir is not None:
#     output_dir = Path(output_dir)
#     if not output_dir.exists():
#         output_dir.mkdir()
#     nlp.to_disk(output_dir)
#     print("Saved model to", output_dir)

## Test the saved model

In [ ]:
# print("Loading from", output_dir)
# nlp2 = spacy.load(output_dir)
# for text, _ in TRAIN_DATA:
#     doc = nlp2(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])